# ch7 순환신경망

## 7.1 기본 순환 신경망

순환 신경망에서 일반적으로 사용되는 데이터와 기본 구조
- 시계열 데이터 : 일정 시간 간격으로 배치된 데이터 (온도,주식,신호변화,음성,대화)

순환 신경망은 시퀀스 데이터를 예측하기 위해 만들어진 모델
- 이전 단계에서 계산된 정보를 가공해서 다음 단계의 계산에 반영한다.

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('./data/kospi.csv')
scaler = MinMaxScaler()
df[['open','high','low','close','volume']] = scaler.fit_transform(df[['Open','High','Low','Volume']])

In [ ]:
df.head()

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')

In [4]:
def seq_data(x,y,sequence_length):
    x_seq = []
    y_seq = []
    for i in range(len(x)-sequence_length):
        x_seq.append(x[i:i+sequence_length])
        y_seq.append(y[i+sequence_length])
    return torch.FloatTensor(x_seq).to(device), torch.FloatTensor(y_seq).to(device).view(-1,1)

In [ ]:
split = 200
sequence_length = 5
x_seq, y_seq = seq_data(X,y,sequence_length)
x_train_seq = x_seq[:split]
y_train_seq = y_seq[:split]
x_test_seq = x_seq[split:]
y_test_seq = y_seq[split:]
print(x_train_seq.size(), y_train_seq.size())
print(x_test_seq.size(), y_test_seq.size())

In [ ]:
train = torch.utils.data.TensorDataset(x_train_seq, y_train_seq)
test = torch.utils.data.TensorDataset(x_test_seq, y_test_seq)
batch_size = 20
train_loader = torch.utils.data.DataLoader(dataset=train, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset = test, batch_size = batch_size)

### RNN 구축에 필요한 하이퍼파라미터 정의

In [ ]:
input_size = x_seq.size(2)
num_layers =2
hidden_size = 8

### RNN 구축하기

In [5]:
class VanillaRNN(nn.Module):

    def __init__(self,input_size,hidden_size,sequence_length,num_layers,device):
        super(VanillaRNN,self).__init__()
        self.device = device
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first= True)
        self.fc = nn.Sequential(nn.Linear(hidden_size*sequence_length, 1), nn.Sigmoid())

    def forward(self,x):
        h0 = torch.zeros(self.num_layers, x.size()[0], self.hidden_size).to(self.device)
        out, _ = self.rnn(x, h0) # _ : 함수의 반환값을 받지 않을 때 사용
        out = out.reshape(out.shape[0],-1)
        out = self.fc(out)
        return out

### 모델 불러오기

In [ ]:
model = VanillaRNN(input_size=input_size,
                   hidden_size=hidden_size,
                   sequence_length=sequence_length,
                   num_layers=num_layers,
                   device=device.to(device))

### 손실함수 및 최적화 방법 정의

In [ ]:
criterion = nn.MSELoss()
num_epochs = 301
optimizer = optim.Adam(model.parameters(), lr=1e-3)

### 모델 학습하기

In [ ]:
loss_graph = []
n = len(train_loader)

for epoch in range(num_epochs):
    running_loss = 0.0

    for data in train_loader:

        seq, target = data # 배치 데이터
        out = model(seq) # 출력값 산출
        loss = criterion(out, target) # 손실함수 계산

        optimizer.zero_grad()
        loss.backward()
        optim.step() # 최적화
        running_loss += loss.item()

    loss_graph.append(running_loss/n)
    if epoch % 100 == 0:
        print('[epoch: %d] loss: %.4f' %(epoch, running_loss/n))

### 학습 손실함수 그리기

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(loss_graph)
plt.show()

### 주가 그리기

In [ ]:
concatdata = torch.utils.data.ConcatDataset([train,test])
data_loader = torch.utils.data.DataLoader(dataset=concatdata, batch_size=100)
with torch.no_grad():
    pred =[]
    model.eval()
    for data in data_loader:
        seq, target = data
        out = model(seq)
        pred += out.cpu().tolist()


plt.figure(figsize=(20,10))
plt.plot(np.ones(100)*len(train),np.linespace(0,1,100),linewidth=0.6)
plt.plot(df['Close'][sequence_length:].values,'--')
plt.plot(pred,'b', linewidth=0.6)
plt.legend(['train boundary','actual', 'prediction'])
plt.show()

## 7.2 LSTM 과 GRU

### 7.2.1 기본 RNN의 문제 
- 기울기 사라짐 
- 장기의존성

### 7.2.2 LSTM

In [ ]:
# 모델 구축하기 

class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, sequence_length, num_layers, device):
        super(LSTM, self).__init__()
        self.device = device
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size*sequence_length,1)


    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x_size()[0], self.hidden_size.to(self.device))
        c0 = torch.zeros(self.num_layers, x_size()[0], self.hidden_size.to(self.device))
        out, _ = self.lstm(x, (h0, c0))
        out = out.reshape(out.shape[0],-1)
        out = self.fc(out)
        return out

### 7.2.3 GRU

In [ ]:
class GRU(nn.Moudule):
    def __init__(self, input_size, hidden_size, sequence_length, num_layers, device):
        super(GRU,self).__init__()
        self.device = device
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first = Ture)
        self.fc = nn.Linear(hidden_size*sequence_length,1)

    def forward(self,x):
        h0 = torch.zeros(self.num_layers, x_size(0), self.hidden_size).to(self.device)
        out, _ =self.gru(x,h0)
        out = out.reshape(out.shape[0],-1)
        out = self.fc(out)
        return out